# TEXT MINING for PRACTICE: 키워드 추출 및 문서요약 
- gensim 패키지를 이용하여 TextRank 기반의 키워드추출/문서요약 알고리즘을 테스트해보자.
---

## 1. Install gensim and newspaper3k

In [1]:
import sys
!{sys.executable} -m pip install gensim newspaper3k

    100% |████████████████████████████████| 24.7MB 752kB/s ta 0:00:011    76% |████████████████████████▍       | 18.8MB 11.3MB/s eta 0:00:01
    100% |████████████████████████████████| 215kB 9.2MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 10.0MB/s ta 0:00:01
    100% |████████████████████████████████| 7.4MB 2.3MB/s ta 0:00:011    12% |████                            | 952kB 17.3MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 8.8MB/s eta 0:00:01
    100% |████████████████████████████████| 194kB 7.9MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 9.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/83/94/7179c3832a6d45b266ddb2aac329e101367fbdb11f425f13771d27f225bb/jmespath-0.9.4-py2.py3-none-any.whl
    100% |████████████████████████████████| 5.4MB 2.2MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/16/8a/1fc3dba0c4923c2a76e1ff0d52b305c44606da63f718d14d3231e21c51b0/s3transfer-0.2.1-py2.p

## 2. 분석할 텍스트 데이터 스크래핑 using newspaper3k

In [13]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
from newspaper import Article

url = 'https://sports.news.naver.com/news.nhn?oid=241&aid=0002977763'
news = Article(url, language='ko')
news.download()
news.parse()
#원문 출력
print(news.text)

사진=안드레 고메스 SNS 사진=안드레 고메스 SNS

참 다행이다. 안드레 고메스(에버턴)와 손흥민(토트넘) 모두 다행이다.지난 4일 안타까운 일이 발생했다. 잉글랜드 프리미어리그 11라운드 토트넘과 에버턴의 경기에서 후반 33분 손흥민은 고메스에 백태클을 가하면서 퇴장을 당했다. 이후 고메스는 토트넘 수비수 서지 오리에와 충돌하면서 쓰러졌다. 발목 골절이라는 큰 부상을 당했다. 불운이 겹쳤고, 고메스와 손흥민 모두 지울 수 없는 상처를 받아야 했다. 이 사태의 후폭풍에 대한 걱정도 컸다. 전 세계 많은 이들이 두 선수를 향해 위로와 응원의 메시지를 보냈다.다행스럽게도 고메스의 수술 결과가 좋게 나왔다. 에버턴은 5일 공식 홈페이지를 통해 "고메스가 오른쪽 발목 수술을 받았고, 매우 잘 끝났다. 병원에서 회복을 한 뒤 곧바로 재활에 들어갈 예정"이라고 발표했다. 경과가 좋아 완벽한 회복에 기대를 걸고 있다. 고메스의 축구 인생도 계속될 수 있다는 희망적인 메시지가 전해졌다. 그리고 7일. 고메스는 자신의 SNS 영상을 통해 "많은 응원을 보내줘 고맙다"라는 마음을 전했다. 영상 속 고메스는 밝은 표정을 지었다. 그의 미소가 곧 희망의 메시지다.다행스럽게도 손흥민도 정신적 충격에서 조금은 벗어난 모습이다. 현지 언론들은 7일 열린 유럽축구연맹(UEFA) 챔피언스리그(UCL) B조 4차전 토트넘과 츠르베나 즈베즈다(세르비아) 경기에 손흥민이 출전하지 않을 가능성을 보도했다. 손흥민에게 안정이 필요하다는 이유에서였다. 하지만 손흥민은 경기에 출전했다. 그리고 유럽 통산 122호, 123호골을 터뜨렸고, 토트넘은 4-0 대승을 거뒀다. 손흥민이 차범근(통산 121골)을 넘어 역대 한국인 유럽 최다골 신기록을 갈아치우는 역사적인 순간. 이때 손흥민은 고메스를 향한 미안함을 먼저 표현했다. 첫 골을 넣은 뒤 손흥민은 두 손을 모아 사과의 뜻을 전했다. 현지 언론들은 감동적 세리머니였다고 표현했다. 경기 후 손흥민은 현지 언론을 통해 "힘든 기간이었다. 하지만 응원해주는

## 3. Tokenizer 정의

In [16]:
from konlpy.tag import Okt
okt = Okt()
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Verb"], stopword=[]):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]


/Users/seojungwon/anaconda3/lib/python3.6/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [20]:
tokenized_text = " ".join(tokenizer(news.text))
tokenized_text

'사진 안드레 고메스 사진 안드레 고메스 차다 다행 안드레 고메스 에버턴 오다 손흥민 토트넘 모두 다행 지난 발생 하다 잉글랜드 프리미어리그 라운드 토트넘 에버턴 경기 후반 손흥민 고메스 태클 가하다 퇴장 이후 고메스 토트넘 수비수 서지 오리 오다 충돌 하다 쓰러지다 발목 골절 크다 부상 불운 치다 고메스 손흥민 모두 지우다 상처 받다 하다 사태 폭풍 대한 걱정 크다 세계 선수 하다 위로 응원 메시지 보내다 다행 고메스 수술 결과 나오다 에버턴 공식 홈페이지 통해 고메스 오른쪽 발목 수술 받다 매우 자다 끝나다 병원 회복 하다 곧바로 재활 들어가다 예정 발표 하다 경과 회복 기대 걸다 고메스 축구 인생 계속 되다 희망 메시지 전해지다 고메스 자신 영상 통해 응원 보내다 마음 하다 영상 고메스 밝다 표정 지다 미소 희망 메시지 다행 손흥민 정신 충격 조금 벗어나다 모습 현지 언론 열리다 유럽 축구 연맹 챔피언스리그 차다 토트넘 츠르베 즈베즈다 세르비아 경기 손흥민 출전 하다 않다 가능성 보도 하다 손흥민 안정 이유 이다 손흥민 경기 출전 하다 유럽 통산 호골 터뜨리다 토트넘 대승 거두다 손흥민 차범근 통산 넘다 역대 한국인 유럽 신기록 치우다 역사 순간 손흥민 고메스 먼저 표현 하다 넣다 손흥민 모으다 사과 하다 현지 언론 감동 세리머니 이다 표현 하다 경기 손흥민 현지 언론 통해 기간 이다 응원 해주다 동료 행운 가지다 깨달 기간 하다 고메스 정말 밝히다 진심 품다 축구 선수 로서 최선 하다 트라우마 벗어나다 위해 축구 집중 하다 역시 희망 메시지 최용재 기자'

## 4. 키워드 추출

In [19]:

print(keywords(tokenized_text).split('\n'))

['손흥민 토트넘', '고메스', '메시지', '에버턴']


## 5. 문서요약

In [26]:
print(summarize(news.text))
print("-"*50)
print(summarize(news.text, word_count=50))
print("-"*50)
print(summarize(news.text, ratio=0.1))


안드레 고메스(에버턴)와 손흥민(토트넘) 모두 다행이다.지난 4일 안타까운 일이 발생했다.
잉글랜드 프리미어리그 11라운드 토트넘과 에버턴의 경기에서 후반 33분 손흥민은 고메스에 백태클을 가하면서 퇴장을 당했다.
그의 미소가 곧 희망의 메시지다.다행스럽게도 손흥민도 정신적 충격에서 조금은 벗어난 모습이다.
현지 언론들은 7일 열린 유럽축구연맹(UEFA) 챔피언스리그(UCL) B조 4차전 토트넘과 츠르베나 즈베즈다(세르비아) 경기에 손흥민이 출전하지 않을 가능성을 보도했다.
하지만 손흥민은 경기에 출전했다.
첫 골을 넣은 뒤 손흥민은 두 손을 모아 사과의 뜻을 전했다.
--------------------------------------------------
안드레 고메스(에버턴)와 손흥민(토트넘) 모두 다행이다.지난 4일 안타까운 일이 발생했다.
잉글랜드 프리미어리그 11라운드 토트넘과 에버턴의 경기에서 후반 33분 손흥민은 고메스에 백태클을 가하면서 퇴장을 당했다.
그의 미소가 곧 희망의 메시지다.다행스럽게도 손흥민도 정신적 충격에서 조금은 벗어난 모습이다.
하지만 손흥민은 경기에 출전했다.
첫 골을 넣은 뒤 손흥민은 두 손을 모아 사과의 뜻을 전했다.
--------------------------------------------------
안드레 고메스(에버턴)와 손흥민(토트넘) 모두 다행이다.지난 4일 안타까운 일이 발생했다.
잉글랜드 프리미어리그 11라운드 토트넘과 에버턴의 경기에서 후반 33분 손흥민은 고메스에 백태클을 가하면서 퇴장을 당했다.
하지만 손흥민은 경기에 출전했다.
